# Load the required modules

In [1]:

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split


# load the data

In [2]:

transactions = pd.read_csv('transaction_data.csv')

In [3]:
data = transactions

In [4]:
customer = data['customerid'].unique()

In [5]:
customers = pd.DataFrame(customer, columns=['customerid'])

# create dummy data

In [6]:

def create_data_dummy(datas):
    data_dummy = datas.copy()
    data_dummy ['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)
    

In [7]:
data_dummy

,customerid,productid,purchase_count,purchase_dummy
0,11000,344,1,1
1,11000,353,1,1
2,11000,485,1,1
3,11000,488,1,1
4,11000,530,1,1
5,11000,541,1,1
6,11000,573,1,1
7,11000,214,1,1
8,11001,217,1,1
9,11001,350,1,1


# Normalise item values across users

In [8]:

df_matrix = pd.pivot_table(data, values='purchase_count', index ='customerid', columns='productid')

In [9]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

# create a table for input to the modeling

In [10]:

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchased-freq']
data_norm = pd.melt(d,id_vars=['customerid'],
value_name ='scaled_purchase_freq').dropna()

print(data_norm.shape)
data_norm.head()

(40031, 3)


,customerid,productid,scaled_purchase_freq
0,11000,214,0.0
4,11004,214,0.0
7,11007,214,0.0
8,11008,214,0.0
19,11019,214,0.0


# Split data into training and testing set

In [11]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [12]:
data

,customerid,productid,purchase_count
0,11000,344,1
1,11000,353,1
2,11000,485,1
3,11000,488,1
4,11000,530,1
5,11000,541,1
6,11000,573,1
7,11000,214,1
8,11001,217,1
9,11001,350,1


In [13]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [14]:
# constant variables to define field names include:
user_id = 'customerid'
item_id = 'productid'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

# training data using 3 different algorithms

In [15]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                        user_id=user_id, 
                                                        item_id=item_id, 
                                                        target=target, 
                                                        similarity_type='pearson')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

# training data in each target using popularity algorithm

In [16]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 47240 observations with 17741 users and 158 items.

Data prepared in: 0.0748s

47240 observations to process; with 158 unique items.

recommendations finished on 1000/18484 queries. users per second: 79107.7

...

recommendations finished on 10000/18484 queries. users per second: 232126

+------------+-----------+--------------------+------+
| customerid | productid |       score        | rank |
+------------+-----------+--------------------+------+
|   11000    |    480    | 1.086067319982957  |  1   |
|   11000    |    529    | 1.0600109110747409 |  2   |
|   11000    |    528    | 1.046402724563644  |  3   |
|   11000    |    539    | 1.0446927374301676 |  4   |
|   11000    |    477    | 1.043200486766048  |  5   |
|   11000    |    222    | 1.0374923171481254 |  6   |
|   11000    |    484    | 1.0363372093023255 |  7   |
|   11000    |    535    | 1.0308823529411764 |  8   |
|   11000    |    217    | 1.0291201982651796 |  9   |
|   11000    |    225    | 1.0267538644470868 |  10  |
|   11001    |    480    | 1.086067319982957  |  1   |
|   11001    |    529    | 1.0600109110747409 |  2   |
|   11001    |    528    | 1.046402724563644  |  3   |
|   11001    |    539    | 1.0446927374301676 |  4   |
|   11001    |    477    | 1.043200486766048  |  5   |
|   11001 

recommendations finished on 11000/18484 queries. users per second: 242227

recommendations finished on 12000/18484 queries. users per second: 247173

recommendations finished on 13000/18484 queries. users per second: 244476

In [17]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


recommendations finished on 14000/18484 queries. users per second: 251040

recommendations finished on 15000/18484 queries. users per second: 260060

recommendations finished on 16000/18484 queries. users per second: 262981

recommendations finished on 17000/18484 queries. users per second: 268847

recommendations finished on 18000/18484 queries. users per second: 263273

...

recommendations finished on 15000/18484 queries. users per second: 308128

recommendations finished on 16000/18484 queries. users per second: 301744

recommendations finished on 17000/18484 queries. users per second: 293802

recommendations finished on 18000/18484 queries. users per second: 254004

+------------+-----------+-------+------+
| customerid | productid | score | rank |
+------------+-----------+-------+------+
|   11000    |    477    |  1.0  |  1   |
|   11000    |    484    |  1.0  |  2   |
|   11000    |    465    |  1.0  |  3   |
|   11000    |    478    |  1.0  |  4   |
|   11000    |    333    |  1.0  |  5   |
|   11000    |    536    |  1.0  |  6   |
|   11000    |    579    |  1.0  |  7   |
|   11000    |    390    |  1.0  |  8   |
|   11000    |    540    |  1.0  |  9   |
|   11000    |    217    |  1.0  |  10  |
|   11001    |    537    |  1.0  |  1   |
|   11001    |    484    |  1.0  |  2   |
|   11001    |    465    |  1.0  |  3   |
|   11001    |    333    |  1.0  |  4   |
|   11001    |    536    |  1.0  |  5   |
|   11001    |    579    |  1.0  |  6   |
|   11001    |    390    |  1.0  |  7   |
|   11001    |    488    |  1.0  |  8   |
|   11001    |    540    |  1.0  |  9   |
|   11001    |    214    |  1.0  |  10  |
|   11002    |    465    |  1.0  |

In [18]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 32024 observations with 15312 users and 36 items.

Data prepared in: 0.071042s

32024 observations to process; with 36 unique items.

recommendations finished on 1000/18484 queries. users per second: 151584

...

recommendations finished on 15000/18484 queries. users per second: 268068

recommendations finished on 16000/18484 queries. users per second: 251481

recommendations finished on 17000/18484 queries. users per second: 247036

recommendations finished on 18000/18484 queries. users per second: 246964

+------------+-----------+----------------------+------+
| customerid | productid |        score         | rank |
+------------+-----------+----------------------+------+
|   11000    |    486    | 0.02197802197802198  |  1   |
|   11000    |    539    |         0.02         |  2   |
|   11000    |    540    | 0.016511867905056758 |  3   |
|   11000    |    467    | 0.014450867052023121 |  4   |
|   11000    |    561    | 0.014285714285714285 |  5   |
|   11000    |    463    | 0.013157894736842105 |  6   |
|   11000    |    465    | 0.013089005235602094 |  7   |
|   11000    |    481    | 0.013043478260869565 |  8   |
|   11000    |    491    | 0.012422360248447204 |  9   |
|   11000    |    483    | 0.011857707509881422 |  10  |
|   11001    |    486    | 0.02197802197802198  |  1   |
|   11001    |    539    |         0.02         |  2   |
|   11001    |    540    | 0.016511867905056758 |  3   |
|   11001    |    467    | 0.014450867052023121 |  4   |
|   11001    |    561    | 0.01

# Collaborative filtering models

In [19]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 47240 observations with 17741 users and 158 items.

Data prepared in: 0.078244s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 15000/18484 queries. users per second: 94678.4

recommendations finished on 16000/18484 queries. users per second: 98601.1

recommendations finished on 17000/18484 queries. users per second: 100358

recommendations finished on 18000/18484 queries. users per second: 102528

+------------+-----------+----------------------+------+
| customerid | productid |        score         | rank |
+------------+-----------+----------------------+------+
|   11000    |    480    | 0.10125453770160675  |  1   |
|   11000    |    477    | 0.09121781587600708  |  2   |
|   11000    |    478    | 0.08351795375347137  |  3   |
|   11000    |    528    | 0.08085720986127853  |  4   |
|   11000    |    222    | 0.07712101936340332  |  5   |
|   11000    |    529    | 0.07613804936408997  |  6   |
|   11000    |    225    | 0.07359151542186737  |  7   |
|   11000    |    537    | 0.07198265194892883  |  8   |
|   11000    |    217    | 0.06905513256788254  |  9   |
|   11000    |    487    | 0.058402761816978455 |  10  |
|   11001    |    477    | 0.22427197865077428  |  1   |
|   11001    |    528    | 0.09053138324192592  |  2   |
|   11001    |    214    | 0.08361170121601649  |  3   |
|   11001    |    222    | 0.07805293798446655  |  4   |
|   11001    |    537    | 0.07

In [20]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 47240 observations with 17785 users and 158 items.

Data prepared in: 0.077295s

...

recommendations finished on 15000/18484 queries. users per second: 174754

recommendations finished on 16000/18484 queries. users per second: 170973

recommendations finished on 17000/18484 queries. users per second: 164861

recommendations finished on 18000/18484 queries. users per second: 157451

+------------+-----------+----------------------+------+
| customerid | productid |        score         | rank |
+------------+-----------+----------------------+------+
|   11000    |    477    | 0.07888364046812057  |  1   |
|   11000    |    478    |  0.078419528901577   |  2   |
|   11000    |    480    | 0.07045881450176239  |  3   |
|   11000    |    225    | 0.06927888095378876  |  4   |
|   11000    |    537    | 0.06604594737291336  |  5   |
|   11000    |    222    | 0.06296557188034058  |  6   |
|   11000    |    528    | 0.061501286923885345 |  7   |
|   11000    |    479    | 0.05382126569747925  |  8   |
|   11000    |    487    | 0.05250490456819534  |  9   |
|   11000    |    217    | 0.04830383509397507  |  10  |
|   11001    |    479    | 0.11889280378818512  |  1   |
|   11001    |    528    | 0.07885675877332687  |  2   |
|   11001    |    537    | 0.07849996536970139  |  3   |
|   11001    |    222    | 0.07350622862577438  |  4   |
|   11001    |    214    |  0.0

In [21]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 32024 observations with 15312 users and 36 items.

Data prepared in: 0.060826s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 15000/18484 queries. users per second: 196325

recommendations finished on 16000/18484 queries. users per second: 201646

recommendations finished on 17000/18484 queries. users per second: 204285

recommendations finished on 18000/18484 queries. users per second: 207211

+------------+-----------+-----------------------+------+
| customerid | productid |         score         | rank |
+------------+-----------+-----------------------+------+
|   11000    |    228    |          0.0          |  1   |
|   11000    |    480    |          0.0          |  2   |
|   11000    |    490    |          0.0          |  3   |
|   11000    |    529    |          0.0          |  4   |
|   11000    |    538    |          0.0          |  5   |
|   11000    |    222    |          0.0          |  6   |
|   11000    |    465    |          0.0          |  7   |
|   11000    |    477    |          0.0          |  8   |
|   11000    |    540    |          0.0          |  9   |
|   11000    |    528    |          0.0          |  10  |
|   11001    |    540    |  0.009339610735575357 |  1   |
|   11001    |    529    |  0.00846711794535319  |  2   |
|   11001    |    217    | 0.0075136224428812666 |  3   |
|   11001    |    535    | 0.0074531833330790205 |  4   |
|   11001    |

In [22]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 47240 observations with 17741 users and 158 items.

Data prepared in: 0.076023s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 15000/18484 queries. users per second: 218700

recommendations finished on 16000/18484 queries. users per second: 213981

recommendations finished on 17000/18484 queries. users per second: 211759

recommendations finished on 18000/18484 queries. users per second: 204216

+------------+-----------+--------------------+------+
| customerid | productid |       score        | rank |
+------------+-----------+--------------------+------+
|   11000    |    480    | 1.080391661399314  |  1   |
|   11000    |    529    | 1.0533782179628457 |  2   |
|   11000    |    528    | 1.041674399852322  |  3   |
|   11000    |    477    | 1.0406209467517833 |  4   |
|   11000    |    539    | 1.0384664213357693 |  5   |
|   11000    |    222    | 1.0335896955808759 |  6   |
|   11000    |    484    | 1.0335470413745838 |  7   |
|   11000    |    217    | 1.0264670986686795 |  8   |
|   11000    |    535    | 1.0255460906554668 |  9   |
|   11000    |    225    | 1.0240146435894661 |  10  |
|   11001    |    480    | 1.0833192818953827 |  1   |
|   11001    |    529    | 1.058114546726467  |  2   |
|   11001    |    528    | 1.0443920810814364 |  3   |
|   11001    |    539    | 1.041689804336688  |  4   |
|   11001    |    477    | 1.0415250257462618 |  5   |
|   11001 

In [23]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 47240 observations with 17785 users and 158 items.

Data prepared in: 0.07918s

...

recommendations finished on 15000/18484 queries. users per second: 197371

recommendations finished on 16000/18484 queries. users per second: 199379

recommendations finished on 17000/18484 queries. users per second: 200335

recommendations finished on 18000/18484 queries. users per second: 199263

+------------+-----------+-------+------+
| customerid | productid | score | rank |
+------------+-----------+-------+------+
|   11000    |    477    |  0.0  |  1   |
|   11000    |    484    |  0.0  |  2   |
|   11000    |    465    |  0.0  |  3   |
|   11000    |    478    |  0.0  |  4   |
|   11000    |    333    |  0.0  |  5   |
|   11000    |    536    |  0.0  |  6   |
|   11000    |    579    |  0.0  |  7   |
|   11000    |    390    |  0.0  |  8   |
|   11000    |    540    |  0.0  |  9   |
|   11000    |    217    |  0.0  |  10  |
|   11001    |    537    |  0.0  |  1   |
|   11001    |    484    |  0.0  |  2   |
|   11001    |    465    |  0.0  |  3   |
|   11001    |    333    |  0.0  |  4   |
|   11001    |    536    |  0.0  |  5   |
|   11001    |    579    |  0.0  |  6   |
|   11001    |    390    |  0.0  |  7   |
|   11001    |    488    |  0.0  |  8   |
|   11001    |    540    |  0.0  |  9   |
|   11001    |    214    |  0.0  |  10  |
|   11002    |    465    |  0.0  |

In [24]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 32024 observations with 15312 users and 36 items.

Data prepared in: 0.064855s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 15000/18484 queries. users per second: 240914

recommendations finished on 16000/18484 queries. users per second: 238838

recommendations finished on 17000/18484 queries. users per second: 233703

recommendations finished on 18000/18484 queries. users per second: 217649

+------------+-----------+----------------------+------+
| customerid | productid |        score         | rank |
+------------+-----------+----------------------+------+
|   11000    |    486    | 0.021947981237055186 |  1   |
|   11000    |    539    | 0.018843276500701906 |  2   |
|   11000    |    540    | 0.015141092205441284 |  3   |
|   11000    |    561    | 0.014285106318337577 |  4   |
|   11000    |    467    | 0.014211470956747236 |  5   |
|   11000    |    481    | 0.013025334606999938 |  6   |
|   11000    |    465    | 0.01278133798020049  |  7   |
|   11000    |    463    | 0.01254470599325079  |  8   |
|   11000    |    491    | 0.012365866966129086 |  9   |
|   11000    |    483    | 0.011752374181634344 |  10  |
|   11001    |    486    | 0.024872244714380626 |  1   |
|   11001    |    539    | 0.02477716326713562  |  2   |
|   11001    |    540    | 0.02462709989587097  |  3   |
|   11001    |    535    | 0.018592558402583427 |  4   |
|   11001    |    463    | 0.01

In [25]:
popularity_model =popularity

In [26]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [27]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/8714 queries. users per second: 71638.4

recommendations finished on 2000/8714 queries. users per second: 84122

recommendations finished on 3000/8714 queries. users per second: 91569.5

recommendations finished on 4000/8714 queries. users per second: 95792.3

...

recommendations finished on 3000/6470 queries. users per second: 77827.1

recommendations finished on 4000/6470 queries. users per second: 77569

recommendations finished on 5000/6470 queries. users per second: 78825.8

recommendations finished on 6000/6470 queries. users per second: 77749.4


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.01282843894899537  | 0.009112754839184518 |
|   2    | 0.01924265842349305  | 0.02772576727754472  |
|   3    | 0.01885625965996905  | 0.03978803267829542  |
|   4    | 0.018083462132921155 |  0.0499698240965629  |
|   5    | 0.017496136012364837 |  0.060780157503496   |
|   6    | 0.018031942297784695 | 0.07528188709796141  |
|   7    | 0.01761978361669239  | 0.08638073158165892  |
|   8    |  0.0178323029366306  | 0.10092478104070073  |
|   9    | 0.017757169843723334 |  0.1148145285935085  |
|   10   | 0.017851622874806714 | 0.12979392065945417  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.06158153457025342

Per User RMSE (best)
+------------+------------------------+-------+
| customerid |         

# we select Cosine similarity on Purchase dummy approach as our final model

In [28]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 59051 observations with 18484 users and 158 items.

Data prepared in: 0.085911s

...

recommendations finished on 15000/18484 queries. users per second: 141856

recommendations finished on 16000/18484 queries. users per second: 134157

recommendations finished on 17000/18484 queries. users per second: 132902

recommendations finished on 18000/18484 queries. users per second: 133360

+------------+-----------+----------------------+------+
| customerid | productid |        score         | rank |
+------------+-----------+----------------------+------+
|   11000    |    478    | 0.10096639394760132  |  1   |
|   11000    |    477    |  0.0984770879149437  |  2   |
|   11000    |    225    | 0.08770759403705597  |  3   |
|   11000    |    480    | 0.08636465668678284  |  4   |
|   11000    |    537    | 0.08083739131689072  |  5   |
|   11000    |    528    | 0.07649999111890793  |  6   |
|   11000    |    217    | 0.07149147987365723  |  7   |
|   11000    |    479    | 0.06456053256988525  |  8   |
|   11000    |    487    | 0.06324635446071625  |  9   |
|   11000    |    467    | 0.05609729140996933  |  10  |
|   11001    |    537    | 0.09024447798728943  |  1   |
|   11001    |    214    | 0.08879245519638061  |  2   |
|   11001    |    528    | 0.08163455724716187  |  3   |
|   11001    |    487    | 0.07803999781608581  |  4   |
|   11001    |    222    | 0.07

In [29]:
#csv output file
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(184840, 4)


,customerid,productid,score,rank
0,11000,478,0.100966,1
1,11000,477,0.098477,2
2,11000,225,0.087708,3
3,11000,480,0.086365,4
4,11000,537,0.080837,5


In [30]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerid', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerid').set_index('customerid')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [31]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/18484 queries. users per second: 111334

recommendations finished on 2000/18484 queries. users per second: 145741

recommendations finished on 3000/18484 queries. users per second: 163559

recommendations finished on 4000/18484 queries. users per second: 184749

recommendations finished on 5000/18484 queries. users per second: 202897

...

recommendations finished on 16000/18484 queries. users per second: 198012

recommendations finished on 17000/18484 queries. users per second: 195891

recommendations finished on 18000/18484 queries. users per second: 194719

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(18484, 1)


,recommendedProducts
customerid,
11000,486|539|540|561|467|481|465|463|491|483
11001,486|539|540|535|463|529|231|481|528|467
11002,486|539|540|467|481|465|491|463|483|231
11003,486|539|561|467|540|481|465|491|463|483
11004,486|539|540|561|467|481|465|491|463|483
